##### Import libraries for data handling, similarity computation, model building, and evaluation metrics

In [156]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score,precision_recall_curve
import numpy as np
import torch.optim as optim


In [157]:
# Loading datasets:
# - drug_interaction: drug-drug interaction data across cell lines
# - drug_feature: features representing each unique drug (indexed by drug name)
# - cell_feature: features representing each cell line (indexed by cell line name)

In [158]:
drug_interaction=pd.read_csv("drug_interaction_drugcomb.csv",low_memory=False)
drug_feature=pd.read_csv("drug_feature.csv",index_col=0)
cell_feature=pd.read_csv("final_cell_line_feature.csv",index_col=0)

In [159]:
drug_interaction.shape    # 1,048,575 drug-drug interactions recorded across various cell lines.

(1048575, 18)

In [160]:
drug_interaction.head()

,block_id,drug_row,drug_col,cell_line_name,conc_r_unit,conc_c_unit,css,synergy_zip,synergy_bliss,synergy_loewe,synergy_hsa,ic50_row,ic50_col,ri_row,ri_col,css_row,css_col,S
0,1,5-FU,ABT-888,A2058,uM,uM,30.869,3.865915,6.256584,-2.951386,5.536903,5.126836,3.267734,11.471,-0.441,22.545,39.193,19.839
1,2,5-FU,ABT-888,A2058,uM,uM,27.460,8.247403,12.333896,3.125927,11.614215,5.126836,3.267734,11.471,-0.441,24.135,30.785,16.430
2,3,5-FU,ABT-888,A2058,uM,uM,29.901,6.063440,11.660209,2.452239,10.940528,5.126836,3.267734,11.471,-0.441,25.561,34.241,18.871
3,4,5-FU,ABT-888,A2058,uM,uM,24.016,-4.280231,5.145209,-4.062761,4.425528,5.126836,3.267734,11.471,-0.441,16.661,31.371,12.986
4,5,5-FU,AZD1775,A2058,uM,uM,66.847,12.284698,15.765467,10.409407,18.656340,5.126836,0.266027,11.471,25.164,76.501,57.193,30.212


##### Creating the target label 'synergistic_status' based on 'synergy_zip':
##### If 'synergy_zip' value is greater than 0, label as 1 (synergistic), else 0 (non-synergistic).

In [161]:
drug_interaction['synergistic_status'] = drug_interaction['synergy_zip'].apply(lambda x: 1 if x > 0 else 0)

##### Selecting only essential columns for prediction:
##### 'drug_row', 'drug_col', and 'cell_line_name' define the drug pair and context,
##### while 'synergistic_status' is the target label for synergy prediction.

In [162]:
drug_interaction=drug_interaction[['drug_row','drug_col','cell_line_name','synergistic_status']]

In [163]:
drug_interaction.head()

,drug_row,drug_col,cell_line_name,synergistic_status
0,5-FU,ABT-888,A2058,1
1,5-FU,ABT-888,A2058,1
2,5-FU,ABT-888,A2058,1
3,5-FU,ABT-888,A2058,0
4,5-FU,AZD1775,A2058,1


In [164]:
drug_interaction.isnull().sum()

drug_row                   0
drug_col              582542
cell_line_name             0
synergistic_status         0
dtype: int64

##### 'drug_col' is essential for defining a drug pair interaction.
##### Rows with missing 'drug_col' cannot represent valid interactions and must be dropped.

In [165]:
drug_interaction.dropna(inplace=True)

In [166]:
drug_interaction.isnull().sum()

drug_row              0
drug_col              0
cell_line_name        0
synergistic_status    0
dtype: int64

In [167]:
drug_interaction.shape #After preprocessing, 1,048,575 drug-drug interactions across various cell lines remain.

(466033, 4)

In [168]:
drug_interaction.synergistic_status.value_counts()    

synergistic_status
0    258510
1    207523
Name: count, dtype: int64

##### The dataset has 258,510 non-synergistic (label 0) and 207,523 synergistic (label 1) samples.
##### This shows a slight class imbalance (~55% vs ~45%), which should be handled during training.

In [169]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))

In [170]:
print(f"total number of drug participating in interaction is {len(interaction_drug)}")

total number of drug participating in interaction is 4150


In [171]:
drug_feature.head()

,drug,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5-FU,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABT-888,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AZD1775,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BEZ-235,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BORTEZOMIB,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### The dataset includes drug features for 4,051 unique drugs to represent each drug in the interaction pairs 

In [172]:
drug_feature.shape

(4051, 2049)

In [173]:
all_drug_with_feature=set(drug_feature['drug'].unique())

In [174]:
print(f"There are a total of {len(all_drug_with_feature)} features available for different drugs ")

There are a total of 4051 features available for different drugs 


##### Confirming missing feature of drug in drug interaction dataset

In [175]:
if interaction_drug.issubset(all_drug_with_feature):
    print("All interaction drugs are present in the feature set.")
else:
    print(" Some interaction drugs are missing in the feature set.")
    missing = interaction_drug - all_drug_with_feature
    
print(f"drug without available feature in drug interaciton {len(missing)}")

 Some interaction drugs are missing in the feature set.
drug without available feature in drug interaciton 99


##### Removing interactions for drug_interaction involving drugs not in the 4,051-drug feature set to ensure all interactions have corresponding features.

In [176]:
drug_interaction = drug_interaction[
    ~drug_interaction['drug_row'].isin(missing) &
    ~drug_interaction['drug_col'].isin(missing)
].reset_index(drop=True)

In [177]:
interaction_drug= set(drug_interaction['drug_row']).union(set(drug_interaction['drug_col']))
all_drug_with_feature=set(drug_feature['drug'].unique())
if interaction_drug.issubset(all_drug_with_feature):
    print("All interaction drugs are present in the feature set.")
else:
    print(" Some interaction drugs are missing in the feature set.")
    
missing = interaction_drug - all_drug_with_feature
    
print(f"drug without available feature in drug interaciton {len(missing)}")

All interaction drugs are present in the feature set.
drug without available feature in drug interaciton 0


##### Assingning unique id to each drug

In [178]:
drug_feature['drug_id'] = range(len(drug_feature))

In [179]:
drug_feature.head()

,drug,0,1,2,3,4,5,6,7,8,...,2039,2040,2041,2042,2043,2044,2045,2046,2047,drug_id
0,5-FU,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABT-888,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,AZD1775,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,BEZ-235,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,BORTEZOMIB,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


##### Storing drug names and their corresponding IDs in drug_to_id to replace drug names with numeric IDs in the interaction dataset.

In [180]:
drug_to_id = dict(zip(drug_feature['drug'], drug_feature['drug_id']))

In [181]:
drug_to_id

{'5-FU': 0,
 'ABT-888': 1,
 'AZD1775': 2,
 'BEZ-235': 3,
 'BORTEZOMIB': 4,
 'DASATINIB': 5,
 'DINACICLIB': 6,
 'ERLOTINIB': 7,
 'GELDANAMYCIN': 8,
 'L778123': 9,
 'LAPATINIB': 10,
 'MK-2206': 11,
 'MK-4827': 12,
 'MK-5108': 13,
 'MK-8669': 14,
 'MK-8776': 15,
 'PD325901': 16,
 'SORAFENIB': 17,
 'SUNITINIB': 18,
 'TEMOZOLOMIDE': 19,
 'ZOLINZA': 20,
 'OXALIPLATIN': 21,
 'SN-38': 22,
 'TOPOTECAN': 23,
 'CARBOPLATIN': 24,
 'CYCLOPHOSPHAMIDE': 25,
 'DEXAMETHASONE': 26,
 'DOXORUBICIN': 27,
 'ETOPOSIDE': 28,
 'GEMCITABINE': 29,
 'VINORELBINE': 30,
 'METFORMIN': 31,
 'METHOTREXATE': 32,
 'MITOMYCINE': 33,
 'MK-4541': 34,
 'MRK-003': 35,
 'PACLITAXEL': 36,
 'VINBLASTINE': 37,
 'RAMELTEON': 38,
 'BRETYLIUM TOSYLATE': 39,
 '3-AMINO-2-OXAZOLIDINONE': 40,
 'METYROSINE': 41,
 'GABAPENTIN': 42,
 'RUFINAMIDE': 43,
 'FEBUXOSTAT': 44,
 'DUTASTERIDE': 45,
 'PRAMIPEXOLE': 46,
 'VALPROIC ACID': 47,
 'TIZOXANIDE': 48,
 'TESTOLACTONE': 49,
 'ACAMPROSATE': 50,
 'DESOXYCORTICOSTERONE PIVALATE': 51,
 'ALTRETAMI

##### Dropping 'drug_id' and 'drug' columns as the index now represents the drug ID,

In [182]:
drug_feature.drop(columns=['drug_id','drug'],inplace=True)

In [183]:
drug_interaction['drug_row_id'] = drug_interaction['drug_row'].map(drug_to_id)
drug_interaction['drug_col_id'] = drug_interaction['drug_col'].map(drug_to_id)

##### Replacing drug names with their corresponding unique IDs in the drug interaction dataset using the drug_to_id mapping.

In [184]:
drug_interaction.head()

,drug_row,drug_col,cell_line_name,synergistic_status,drug_row_id,drug_col_id
0,5-FU,ABT-888,A2058,1,0,1
1,5-FU,ABT-888,A2058,1,0,1
2,5-FU,ABT-888,A2058,1,0,1
3,5-FU,ABT-888,A2058,0,0,1
4,5-FU,AZD1775,A2058,1,0,2


In [185]:
drug_interaction = drug_interaction.drop(columns=['drug_row', 'drug_col'])
cols = ['drug_row_id', 'drug_col_id'] + [col for col in drug_interaction.columns if col not in ['drug_row_id', 'drug_col_id']]
drug_interaction = drug_interaction[cols]

In [186]:
drug_interaction.head()

,drug_row_id,drug_col_id,cell_line_name,synergistic_status
0,0,1,A2058,1
1,0,1,A2058,1
2,0,1,A2058,1
3,0,1,A2058,0
4,0,2,A2058,1


##### Drug similarity based on feature

In [187]:
cos_sim_matrix = cosine_similarity(drug_feature)

In [188]:
cos_sim_matrix

array([[1.        , 0.17699808, 0.11211037, ..., 0.11646819, 0.09727208,
        0.14808722],
       [0.17699808, 1.        , 0.2073627 , ..., 0.28200837, 0.22898571,
        0.21912525],
       [0.11211037, 0.2073627 , 1.        , ..., 0.17366199, 0.20719896,
        0.34698416],
       ...,
       [0.11646819, 0.28200837, 0.17366199, ..., 1.        , 0.21525296,
        0.27526932],
       [0.09727208, 0.22898571, 0.20719896, ..., 0.21525296, 1.        ,
        0.26000576],
       [0.14808722, 0.21912525, 0.34698416, ..., 0.27526932, 0.26000576,
        1.        ]], shape=(4051, 4051))

In [189]:
cos_sim_matrix.shape

(4051, 4051)

##### Creating an adjacency matrix where edges represent synergistic drug-drug interactions (label = 1).

In [106]:
df_synergy = drug_interaction[drug_interaction['synergistic_status'] == 1]

In [107]:
num_drugs = max(drug_interaction['drug_row_id'].max(), drug_interaction['drug_col_id'].max()) + 1

In [108]:
adj_matrix = np.zeros((num_drugs, num_drugs))

In [109]:
for _, row in df_synergy.iterrows():
    i = int(row['drug_row_id'])
    j = int(row['drug_col_id'])
    adj_matrix[i, j] = 1
    adj_matrix[j, i] = 1  

In [110]:
np.fill_diagonal(adj_matrix, 1)   #adding symmetric representaiton

In [111]:
adj_matrix    #adjancent matrix based in posiive label (value=1) , negative(value =0)

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], shape=(4051, 4051))

##### Preparing drug feature and adjacnet matrix for drug feature compression/extraction

In [112]:
drug_feature=np.array(drug_feature)

In [113]:
X = torch.tensor(drug_feature, dtype=torch.float32)        
A_label = torch.tensor(adj_matrix, dtype=torch.float32)

In [114]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X = X.to(device)
A_label = A_label.to(device)

##### Normalize the adjacency matrix using symmetric normalization: D^(-1/2) * A * D^(-1/2) to prevent high-degree nodes (drugs with many interactions) from dominating the representation.

In [116]:
def normalize_adj(adj):
    adj = adj + torch.eye(adj.shape[0]).to(device)  # Add self-loops
    deg = torch.sum(adj, dim=1)
    deg_inv_sqrt = torch.pow(deg, -0.5)
    deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0.0
    D_inv_sqrt = torch.diag(deg_inv_sqrt)
    return D_inv_sqrt @ adj @ D_inv_sqrt

norm_adj = normalize_adj(A_label)

In [115]:
density = A_label.sum() / (A_label.shape[0] ** 2)
print(f"Sparsity: {density.item():.6f}")

Sparsity: 0.003793


##### Sparsity of the adjacency matrix is 0.003793,
##### meaning only ~0.38% of all possible drug-drug pairs have an edge (i.e., are connected),

#### Graph Autoencoder (GAE) model for learning node embeddings and reconstructing adjacency matrix.

In [62]:
class GAE(torch.nn.Module):
    def __init__(self, in_dim, hidden1, hidden2, dropout=0.3):
        super(GAE, self).__init__()
        self.W1 = torch.nn.Parameter(torch.empty(in_dim, hidden1))
        self.W2 = torch.nn.Parameter(torch.empty(hidden1, hidden2))
        self.decoder_weight = torch.nn.Parameter(torch.empty(hidden2, hidden2))
        torch.nn.init.xavier_uniform_(self.W1)
        torch.nn.init.xavier_uniform_(self.W2)
        torch.nn.init.xavier_uniform_(self.decoder_weight)
        self.dropout = torch.nn.Dropout(dropout)

    def encode(self, X, norm_adj):
        h1 = torch.nn.functional.leaky_relu(norm_adj @ X @ self.W1)
        h1 = self.dropout(h1)
        h2 = torch.nn.functional.leaky_relu(norm_adj @ h1 @ self.W2)
        return h2

    def decode(self, Z):
        return Z @ self.decoder_weight @ Z.T

    def forward(self, X, norm_adj):
        Z = self.encode(X, norm_adj)
        A_pred = self.decode(Z)
        return A_pred, Z




##### Function to create positive edge and sUNIFORM  negative edge sample with respect to positive pair

In [117]:
def get_positive_edges(adj):
    triu_idx = torch.triu_indices(adj.shape[0], adj.shape[0], offset=1)
    mask = adj[triu_idx[0], triu_idx[1]] > 0
    return triu_idx[:, mask]

def sample_negative_edges(adj, count):
    N = adj.shape[0]
    neg = set()
    while len(neg) < count:
        i, j = random.randint(0, N - 1), random.randint(0, N - 1)
        if i != j and adj[i, j] == 0 and (i, j) not in neg and (j, i) not in neg:
            neg.add((i, j))
    return torch.tensor(list(neg), dtype=torch.long).T



##### Evaluation function for prediction  model

In [118]:
def evaluate_edge_predictions(y_true, y_score):
    y_true_np = y_true.cpu().numpy()
    y_score_np = y_score.cpu().numpy()
    roc = roc_auc_score(y_true_np, y_score_np)
    pr = average_precision_score(y_true_np, y_score_np)
    precision, recall, _ = precision_recall_curve(y_true_np, y_score_np)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    best_f1 = f1.max()
    return roc, pr, best_f1

In [131]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X, A_label = X.to(device), A_label.to(device)

# Get positive edges
pos_edges = get_positive_edges(A_label).T.cpu().numpy()  # shape (num_pos, 2)

# Train/test split 
train_pos, test_pos = train_test_split(pos_edges, test_size=0.2, random_state=42)

train_pos = torch.tensor(train_pos.T, dtype=torch.long).to(device)
test_pos = torch.tensor(test_pos.T, dtype=torch.long).to(device)
total_neg_count=train_pos.shape[1]+test_pos.shape[1]

all_neg = sample_negative_edges(A_label, total_neg_count)  

all_neg_np = all_neg.T.cpu().numpy()
train_neg_np, test_neg_np = train_test_split(all_neg_np, test_size=0.2, random_state=42)

train_neg = torch.tensor(train_neg_np.T, dtype=torch.long).to(device)
test_neg = torch.tensor(test_neg_np.T, dtype=torch.long).to(device)



##### model initialization

In [120]:
model = GAE(in_dim=2048, hidden1=512, hidden2=128, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()

norm_adj = normalize_adj(A_label)


In [121]:

def get_loss_and_metrics(model, X, norm_adj, pos_edge_idx, neg_edge_idx):
    model.eval()
    with torch.no_grad():
        A_pred, _ = model(X, norm_adj)
        pos_pred = A_pred[pos_edge_idx[0], pos_edge_idx[1]]
        neg_pred = A_pred[neg_edge_idx[0], neg_edge_idx[1]]
        y_true = torch.cat([torch.ones_like(pos_pred), torch.zeros_like(neg_pred)])
        y_score = torch.cat([pos_pred, neg_pred])
        loss = loss_fn(y_score, y_true)
        roc, pr, best_f1 = evaluate_edge_predictions(y_true, y_score)
    return loss.item(), roc, pr, best_f1

In [135]:
train_pos

torch.Size([23277])

In [139]:
pos_pred.shape

torch.Size([23277])

In [146]:

epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    A_pred, _ = model(X, norm_adj)
    pos_pred = A_pred[train_pos[0], train_pos[1]]
    neg_pred = A_pred[train_neg[0], train_neg[1]]

    y_true = torch.cat([torch.ones_like(pos_pred), torch.zeros_like(neg_pred)])
    y_score = torch.cat([pos_pred, neg_pred])

    loss = loss_fn(y_score, y_true)
    loss.backward()
    optimizer.step()

    train_loss = loss.item()

    test_loss, test_roc, test_pr, test_f1 = get_loss_and_metrics(model, X, norm_adj, test_pos, test_neg)

    print(f"Epoch {epoch+1:03d} | Train Loss: {train_loss:.4f} | "
          f"Test Loss: {test_loss:.4f} | Test ROC-AUC: {test_roc:.4f} | "
          f"Test PR-AUC: {test_pr:.4f} | Test Best-F1: {test_f1:.4f}")

print(f"Test Loss: {test_loss:.4f}")
print(f"Test ROC-AUC: {test_roc:.4f}")
print(f"Test PR-AUC: {test_pr:.4f}")
print(f"Test Best F1: {test_f1:.4f}")


Epoch 001 | Train Loss: 0.7004 | Test Loss: 9.9519 | Test ROC-AUC: 0.9164 | Test PR-AUC: 0.9225 | Test Best-F1: 0.8480
Epoch 002 | Train Loss: 10.0684 | Test Loss: 181.2749 | Test ROC-AUC: 0.0879 | Test PR-AUC: 0.3165 | Test Best-F1: 0.6667
Epoch 003 | Train Loss: 182.6883 | Test Loss: 12.8017 | Test ROC-AUC: 0.0894 | Test PR-AUC: 0.3167 | Test Best-F1: 0.6667
Epoch 004 | Train Loss: 12.9485 | Test Loss: 0.6147 | Test ROC-AUC: 0.9163 | Test PR-AUC: 0.9223 | Test Best-F1: 0.8458
Epoch 005 | Train Loss: 0.6074 | Test Loss: 0.6830 | Test ROC-AUC: 0.3569 | Test PR-AUC: 0.5690 | Test Best-F1: 0.6688
Epoch 006 | Train Loss: 0.6833 | Test Loss: 0.6599 | Test ROC-AUC: 0.8459 | Test PR-AUC: 0.8494 | Test Best-F1: 0.7927
Epoch 007 | Train Loss: 0.6625 | Test Loss: 0.6030 | Test ROC-AUC: 0.9265 | Test PR-AUC: 0.9299 | Test Best-F1: 0.8568
Epoch 008 | Train Loss: 0.6028 | Test Loss: 0.5982 | Test ROC-AUC: 0.9407 | Test PR-AUC: 0.9417 | Test Best-F1: 0.8705
Epoch 009 | Train Loss: 0.5967 | Test Los

In [147]:

epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    A_pred, _ = model(X, norm_adj)
    pos_pred = A_pred[train_pos[0], train_pos[1]]
    neg_pred = A_pred[train_neg[0], train_neg[1]]

    y_true = torch.cat([torch.ones_like(pos_pred), torch.zeros_like(neg_pred)])
    y_score = torch.cat([pos_pred, neg_pred])

    loss = loss_fn(y_score, y_true)
    loss.backward()
    optimizer.step()

    train_loss = loss.item()

    test_loss, test_roc, test_pr, test_f1 = get_loss_and_metrics(model, X, norm_adj, test_pos, test_neg)

    print(f"Epoch {epoch+1:03d} | Train Loss: {train_loss:.4f} | "
          f"Test Loss: {test_loss:.4f} | Test ROC-AUC: {test_roc:.4f} | "
          f"Test PR-AUC: {test_pr:.4f} | Test Best-F1: {test_f1:.4f}")

print(f"Test Loss: {test_loss:.4f}")
print(f"Test ROC-AUC: {test_roc:.4f}")
print(f"Test PR-AUC: {test_pr:.4f}")
print(f"Test Best F1: {test_f1:.4f}")


Epoch 001 | Train Loss: 0.0159 | Test Loss: 0.0334 | Test ROC-AUC: 0.9977 | Test PR-AUC: 0.9956 | Test Best-F1: 0.9939
Epoch 002 | Train Loss: 0.0181 | Test Loss: 0.0340 | Test ROC-AUC: 0.9977 | Test PR-AUC: 0.9955 | Test Best-F1: 0.9940
Epoch 003 | Train Loss: 0.0167 | Test Loss: 0.0366 | Test ROC-AUC: 0.9976 | Test PR-AUC: 0.9954 | Test Best-F1: 0.9944
Epoch 004 | Train Loss: 0.0175 | Test Loss: 0.0389 | Test ROC-AUC: 0.9976 | Test PR-AUC: 0.9954 | Test Best-F1: 0.9938
Epoch 005 | Train Loss: 0.0170 | Test Loss: 0.0400 | Test ROC-AUC: 0.9976 | Test PR-AUC: 0.9955 | Test Best-F1: 0.9936
Epoch 006 | Train Loss: 0.0174 | Test Loss: 0.0392 | Test ROC-AUC: 0.9976 | Test PR-AUC: 0.9955 | Test Best-F1: 0.9935
Epoch 007 | Train Loss: 0.0170 | Test Loss: 0.0373 | Test ROC-AUC: 0.9977 | Test PR-AUC: 0.9956 | Test Best-F1: 0.9935
Epoch 008 | Train Loss: 0.0170 | Test Loss: 0.0351 | Test ROC-AUC: 0.9977 | Test PR-AUC: 0.9956 | Test Best-F1: 0.9936
Epoch 009 | Train Loss: 0.0174 | Test Loss: 0.03

In [149]:
model.eval()
with torch.no_grad():
    A_pred_final, Z_final = model(X, norm_adj)  # [4051×4051], [4051×128]
    A_pred_np = A_pred_final.cpu().numpy()      # Final decoded matrix
    Z_np = Z_final.cpu().numpy() 

In [150]:
import torch.nn as nn

class GeneExpressionAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(19225, 4096),
            nn.ReLU(),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 1024),
            nn.ReLU(),
            nn.Linear(1024, 4096),
            nn.ReLU(),
            nn.Linear(4096, 19225)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

model = GeneExpressionAutoencoder()


In [151]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming your dataframe is called df
cell_names = cell_feature.iloc[:, 0]  # first column, cell line names
gene_features = cell_feature.iloc[:, 1:].values  # numeric matrix (717, 19226)


In [152]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
gene_features_scaled = scaler.fit_transform(gene_features)


In [153]:
X = torch.tensor(gene_features_scaled, dtype=torch.float32)

dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [154]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 500
 
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs = batch[0]
        optimizer.zero_grad()
        outputs, _ = model(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataset):.4f}")


Epoch 1, Loss: 0.9434
Epoch 2, Loss: 0.8882
Epoch 3, Loss: 0.8675
Epoch 4, Loss: 0.8554
Epoch 5, Loss: 0.8439
Epoch 6, Loss: 0.8385
Epoch 7, Loss: 0.8140
Epoch 8, Loss: 0.8026
Epoch 9, Loss: 0.7933
Epoch 10, Loss: 0.7797
Epoch 11, Loss: 0.7754
Epoch 12, Loss: 0.7560
Epoch 13, Loss: 0.7621
Epoch 14, Loss: 0.7555
Epoch 15, Loss: 0.7485
Epoch 16, Loss: 0.7346
Epoch 17, Loss: 0.7182
Epoch 18, Loss: 0.7149
Epoch 19, Loss: 0.7171
Epoch 20, Loss: 0.7149
Epoch 21, Loss: 0.7268
Epoch 22, Loss: 0.6934
Epoch 23, Loss: 0.6984
Epoch 24, Loss: 0.6871
Epoch 25, Loss: 0.6759
Epoch 26, Loss: 0.6721
Epoch 27, Loss: 0.6675
Epoch 28, Loss: 0.6499
Epoch 29, Loss: 0.6470
Epoch 30, Loss: 0.6394
Epoch 31, Loss: 0.6491
Epoch 32, Loss: 0.6423
Epoch 33, Loss: 0.6357
Epoch 34, Loss: 0.6387
Epoch 35, Loss: 0.6328
Epoch 36, Loss: 0.6186
Epoch 37, Loss: 0.6086
Epoch 38, Loss: 0.6017
Epoch 39, Loss: 0.5883
Epoch 40, Loss: 0.5861
Epoch 41, Loss: 0.5863
Epoch 42, Loss: 0.5850
Epoch 43, Loss: 0.5843
Epoch 44, Loss: 0.58

In [193]:
model.eval()
with torch.no_grad():
    X_tensor = torch.tensor(gene_features_scaled, dtype=torch.float32)
    _, encoded_features = model(X_tensor)

print(encoded_features)
np.save('cell_feature.npy', encoded_features)
np.save('drug_feature.npy', Z_np)



tensor([[ 15.1518,   7.7755,  10.9757,  ..., -16.5542, -17.2571,   2.9717],
        [-28.7919,  32.3704, -10.2417,  ..., -10.8385, 100.3518, -24.1120],
        [-27.6983,  24.7137,  -4.8231,  ...,  24.4822,   6.4458, -53.7301],
        ...,
        [ 72.4802,  55.8659,   8.6138,  ..., -15.4404,  24.4700, -25.8785],
        [ 83.8114,  58.3860,   4.7625,  ...,  30.9066,  33.4361,  11.6774],
        [ 30.1632,   1.6251,  -3.5079,  ..., -33.6430, -41.2060, -41.8085]])
